In [ ]:
#Importing required libraries
import requests
import csv
import numpy as np
import pandas as pd
from datetime import date
import glob
import os
import json
import time

In [ ]:
# Collecting Crime Data for each day and converting to CSV file


crime_response = requests.get("https://phl.carto.com/api/v2/sql?filename=incidents_part1_part2&format=csv&q=SELECT%20*%20,%20ST_Y(the_geom)%20AS%20lat,%20ST_X(the_geom)%20AS%20lng%20FROM%20incidents_part1_part2%20WHERE%20dispatch_date_time%20%3E=%20%272022-01-01%27%20AND%20dispatch_date_time%20%3C%20%272023-01-01%27")
crime_text = crime_response.text.strip().split("\n")
crime_reader = csv.reader(crime_text)
crime_list=list(crime_reader)

#REMOVING USELESS INFORMATION AND CLEANING ROWS


newarr=[]
for list_item in crime_list[1:]:
    if list_item[14] == '' or list_item[15] == '' or list_item[16] =='' or list_item[17] == '':
        continue
    else:
        newarr.append(list_item)
headings = crime_list[:1][0]
newarr.insert(0,headings)
df = pd.DataFrame(newarr[1:], columns=headings)

df.drop(['the_geom','cartodb_id','the_geom_webmercator','objectid','dc_dist','psa','dispatch_date_time','dc_key','ucr_general','point_x','point_y'], axis = 1, inplace = True)
today = date.today()
d1 = today.strftime("%d/%m/%Y")
fileName = str(d1) + ".csv"
df.to_csv("./Crime/11-24-22.csv")

In [ ]:
#list all csv files only
csv_files = glob.glob('*.{}'.format('csv'))
csv_files

In [ ]:
#Reading all the csv files and appending to a dataframe

df_append = pd.DataFrame()
#append all files together
for file in csv_files[:-1]:
            df_temp = pd.read_csv(file)
            df_append = df_append.append(df_temp, ignore_index=True)
df_append

In [ ]:
#Dropping duplicate elements and columns
df_append = df_append.drop_duplicates()
df_append = df_append.drop(['Unnamed: 0','hour_'],axis=1)

In [ ]:
#Sorting the dataframe based on the date the crime happened
df_append = df_append.sort_values(by=['dispatch_date'],ignore_index=True)
df_append

In [ ]:
#Converting Dataframe to csv file

df_append.to_csv("Cleaned-Crime.csv")

In [ ]:
#Reading Crime Dataset

csv_files = glob.glob('*.{}'.format('csv'))
crimeDataFile = csv_files[-2]

crimeData = pd.read_csv(crimeDataFile)

requiredDates=list(set(crimeData['dispatch_date'].tolist()))
requiredDates.sort()
requiredDates

In [ ]:
#Collecting Weather Data

weather_details = {}



def get_weather(date):
    
    weather_response = requests.get('http://api.weatherapi.com/v1/history.json?key=2eec0b76d1b24fe28c5164334222511&q=Philadelphia&dt='+date)
    weather_data = weather_response.json()
    dailyForecast = weather_data['forecast']['forecastday'][0]['day']
    astro = weather_data['forecast']['forecastday'][0]['astro']
    hourlyForecast = weather_data['forecast']['forecastday'][0]['hour']
    weather_details[date] = [dailyForecast,astro,hourlyForecast]
        
        
    

for date in requiredDates:
    get_weather(date)
    
weather_details

In [ ]:
updated_weather_details={}
for i in weather_details:
    item = weather_details[i]
    dailyForecast = item[0]
    astro = item[1]
    hourlyForecast = item[2]
    
    #Daily Forecast
    maxtemp_f = dailyForecast['maxtemp_f']
    mintemp_f = dailyForecast['mintemp_f']
    avgtemp_f = dailyForecast['avgtemp_f']
    maxwind_mph = dailyForecast['maxwind_mph']
    avghumidity = dailyForecast['avghumidity']
    totalprecip_mm = dailyForecast['totalprecip_mm']
    condition = dailyForecast['condition']['text']
    
    forecastDict = {'maxTemp' : maxtemp_f,
                'minTemp' : mintemp_f,
                'avgTemp' : avgtemp_f,
                'maxWind' : maxwind_mph,
                'avgHumidity' : avghumidity,
                'totalPrecipitation':totalprecip_mm,
                'condition': condition
               }
    
    
    #Astro
    sunrise = astro['sunrise']
    sunset = astro['sunset']
    moonrise = astro['moonrise']
    moonset = astro['moonset']
    moonphase = astro['moon_phase']
    
    astroDict = {
            'sunrise' : sunrise,
            'sunset' : sunset,
            'moonrise' : moonrise,
            'moonset' : moonset,
            'moonphase' : moonphase
    } 
    
    #Hourly Forecast
    
    updated_weather_details[i] = [forecastDict, astroDict,hourlyForecast]

    
updated_weather_details  
    

In [ ]:
weather = pd.DataFrame()
# stockDF = pd.DataFrame(columns)


df = pd.DataFrame(crimeData)
time = df['dispatch_time'].tolist()
headings = ['Temperature','Maximum Temperature', 'Average Temperature','Maximum Wind','Average Humidity','Total Precipitation', 'Condition','Sunrise', 'Sunset','Moonrise','Moonset','Moon Phase']

weather = pd.DataFrame(columns=headings)

for j, ele in enumerate(crimeData['dispatch_date']):
    
    weather_info = updated_weather_details[ele]
    
    daily = weather_info[0]
    astro = weather_info[1]
    hourly = weather_info[2]
    time = int(crimeData.iloc[j]['dispatch_time'][:2])
    
    todayData = hourly[time]
    
    temp = todayData['temp_f']
    
    
    
    
    values = {'Temperature' : temp
              ,'Maximum Temperature' : daily['maxTemp'],
              'Average Temperature' : daily['avgTemp'],
              'Maximum Wind' : daily['maxWind'],
              'Average Humidity' : daily['avgHumidity'],
              'Total Precipitation' : daily['totalPrecipitation'],
              'Condition' : todayData['condition']['text'],
              'Sunrise' : astro['sunrise'],
              'Sunset' : astro['sunset'],
              'Moonrise' : astro['moonrise'],
              'Moonset' : astro['moonset'],
              'Moon Phase' : astro['moonphase']
             }

    information = pd.DataFrame([values],columns=headings)
    weather = weather.append(information,ignore_index = True)
    

weather

In [ ]:
weather.to_csv("Weather.csv")

In [ ]:
#Merging Crime and Weather Data

weatherData = pd.read_csv("Weather.csv")
crimeWeather = pd.concat([crimeData, weatherData], axis=1)


crimeWeather = crimeWeather.drop(['Unnamed: 0'],axis=1)
crimeWeather[:10]

In [ ]:
crimeWeather.to_csv("Crime-Weather.csv")

In [ ]:
#Stock Market Data

dailyData = []
# url = 'https://api.twelvedata.com/time_series?symbol=AAPL,EUR/USD,ETH/BTC:Huobi,TRP:TSX&interval=1day&apikey=fc815ba85f984ee3bca4d9d1dd6d8524'
url = 'https://api.twelvedata.com/time_series?&start_date=2022-01-01&end_date=2022-11-23&symbol=aapl&interval=1day&apikey=fc815ba85f984ee3bca4d9d1dd6d8524'


In [ ]:
stockData = requests.get(url).json()
# stockData
dailyData =  stockData['values']
sortedData = sorted(dailyData, key=lambda d: d['datetime']) 
sortedData

In [ ]:

#Holiday Data


holidayUrl = 'https://holidayapi.com/v1/holidays?pretty&key=e1df5793-02a5-4982-bf6d-df30497e8305&country=US&year=2021'


In [ ]:
holidayResponse = requests.get(holidayUrl).json()

In [ ]:
holidays = holidayResponse['holidays']

holidays

def filteredHolidays(holiday):
        
    date = holiday['date']
    dateArray = date.split("-")
    year = int(dateArray[0])+1
    newDate = str(year)+'-'+dateArray[1]+'-'+dateArray[2]
    public = "NOT A PUBLIC HOLIDAY"
    
    if(holiday['public']==True):
        public = "PUBLIC HOLIDAY"
        
        
    updatedHolidays = {
        'Holiday' : holiday['name'],
        'Date' : newDate,
        'Public Holiday' : public,
    }
    
    return updatedHolidays

updatedHolidays = map(filteredHolidays,holidays)

updatedHolidays = list(updatedHolidays)

In [ ]:
holidayDataFrame = pd.DataFrame(updatedHolidays)

holidayDataFrame = holidayDataFrame[:-1]
# holidayDataFrame
updatedHolidays

In [ ]:
#Getting only stock dates as an array

def stock_dates(stock_dict):
    return stock_dict['datetime']
    
    
    
stockDates = list(map(stock_dates,sortedData))
stockDates

In [ ]:
#Getting only Holiday Dates as an array

def holiday_dates(holidayDict):
    return holidayDict['Date']

holidayDates = list(map(holiday_dates,updatedHolidays))
holidayDates

In [ ]:
#Getting a certain dictionary from stock data

def get_stock_dict(d):
    
    for data in sortedData:
        if data['datetime'] == str(d):
            return data
        else:
            continue
    
    dict = {
                'datetime':str(d),
                'open':0,
                'high':0,
                'low':0,
                'close':0,
                'volume':0
            }
    return dict
get_stock_dict('2022-01-05')

In [ ]:
#Getting a certain dictionary from  holidays

def get_holiday_Data(d):
    
    for data in updatedHolidays:
        if data['Date'] == str(d):
            return data
        else:
            continue
            
    value = {'Holiday': "NOT A HOLIDAY",
              'Date': str(d),
             'Public Holiday': 'NOT A HOLIDAY'
                    }
    return value
get_holiday_Data('2022-01-13')

In [ ]:
crimeWeather = pd.read_csv("Crime-Weather.csv")
crimeWeather = crimeWeather.drop(['Unnamed: 0'],axis=1)

holidayHeading = ['Holiday','Date','Public Holiday']
stockHeading = ['datetime','open','high','low','close','volume']
stockDF = pd.DataFrame(columns = stockHeading)
holidayDF = pd.DataFrame(columns = holidayHeading)
for j, ele in enumerate(crimeData['dispatch_date']):

    stockValue = get_stock_dict(ele)
    stock_temp = pd.DataFrame([stockValue],columns = stockHeading)
    
    stockDF = stockDF.append(stock_temp,ignore_index=True)
#     pd.concat([stockDF,stock_temp],ignore_index = True)

stockDF


In [ ]:
for j, ele in enumerate(crimeData['dispatch_date']):
    
    holidayValue = get_holiday_Data(ele)
    holiday_temp = pd.DataFrame([holidayValue],columns = holidayHeading)
    holidayDF = holidayDF.append(holiday_temp,ignore_index=True)

holidayDF

In [ ]:
stockDF.to_csv("Stock.csv")

In [ ]:
holidayDF.to_csv("Holiday.csv")

In [ ]:
#Merging Crime-Weather, Stock and Holiday Dataset


finalCW = pd.read_csv("Crime-Weather.csv")
holidayCSV = pd.read_csv("Holiday.csv")
stockCSV = pd.read_csv("Stock.csv")
mergedDataset = pd.concat([finalCW, holidayCSV,stockCSV], axis=1)

mergedDataset = mergedDataset.drop(['Unnamed: 0','datetime','Date'],axis=1)
my_list = mergedDataset.columns.values.tolist()
# mergedDataset[:10]
my_list

In [ ]:
#Renaming columns
updated_columns = {'dispatch_date':'Date', 
                   'dispatch_time':'Dispatch Time',
                   'location_block':'Location',
                   'text_general_code' : 'Offense',
                    'lat' : 'Latitude',
                   'lng' : 'Longitude',
                   'open' : 'Open',
                   'high': 'High',
                   'low' : 'Low',
                   'close' : 'Close',
                   'volume' : 'Volume'
                  
                  }
mergedDataset.rename(columns =updated_columns , inplace = True)



In [ ]:
mergedDataset[:100]

In [ ]:
#Converting dataframe to CSV


mergedDataset.to_csv("Final Dataset.csv")